# Healthcare Quality Metrics Dashboard (HEDIS-Style)

**Summary:**  
Built an end-to-end healthcare quality analytics project using synthetic, publication-safe data.  
Modeled HEDIS-style measures with numerator/denominator logic, stratified results to surface equity considerations, visualized insights in Python, and queried results using SQL (SQLite).

**Key Skills:**  
Healthcare Analytics · SQL · Python · Data Quality · HEDIS-style Measures · Data Visualization · Health Equity

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path

# ----------------------------
# Config
# ----------------------------
SEED = 42
N_PATIENTS = 900
MEASUREMENT_YEAR = 2024
OUT_DIR = Path("synthetic_quality_project")
OUT_DIR.mkdir(exist_ok=True)

rng = np.random.default_rng(SEED)

start_year = pd.Timestamp(f"{MEASUREMENT_YEAR}-01-01")
end_year = pd.Timestamp(f"{MEASUREMENT_YEAR}-12-31")

def random_dates(n, start, end, rng):
    """Return n random dates between start and end (inclusive)."""
    start_u = start.value // 10**9
    end_u = end.value // 10**9
    secs = rng.integers(start_u, end_u + 1, size=n)
    # Convert to Series so .dt works reliably
    return pd.Series(pd.to_datetime(secs, unit="s")).dt.floor("D")

# ----------------------------
# 1) Patients
# ----------------------------
patient_id = np.arange(1, N_PATIENTS + 1)

# Age distribution: skew adult-heavy with a tail
ages = np.clip(rng.normal(loc=45, scale=18, size=N_PATIENTS).round().astype(int), 0, 90)
birth_years = MEASUREMENT_YEAR - ages
birth_dates = pd.to_datetime(
    [f"{y}-{rng.integers(1,13):02d}-{rng.integers(1,29):02d}" for y in birth_years]
)

sex = rng.choice(["Female", "Male"], size=N_PATIENTS, p=[0.52, 0.48])

race = rng.choice(
    ["White", "Black", "Hispanic", "Asian", "Other"],
    size=N_PATIENTS,
    p=[0.50, 0.18, 0.20, 0.08, 0.04],
)

ethnicity = np.where(race == "Hispanic", "Hispanic", rng.choice(["Non-Hispanic", "Hispanic"], size=N_PATIENTS, p=[0.88, 0.12]))

zip_code = rng.choice(
    ["08540", "08608", "08016", "07102", "08901", "10001", "19104"], size=N_PATIENTS
)

active_flag = rng.choice([True, False], size=N_PATIENTS, p=[0.92, 0.08])

patients = pd.DataFrame(
    {
        "patient_id": patient_id,
        "birth_date": birth_dates,
        "sex": sex,
        "race": race,
        "ethnicity": ethnicity,
        "zip_code": zip_code,
        "active_flag": active_flag,
    }
)

# ----------------------------
# 2) Encounters
# ----------------------------
# Encounters per patient: 0–6, more for older ages
base_visits = rng.poisson(lam=2.0, size=N_PATIENTS)
age_bump = (ages >= 55).astype(int) + (ages >= 70).astype(int)
n_visits = np.clip(base_visits + age_bump, 0, 7)

enc_rows = []
enc_id = 1
for pid, v in zip(patient_id, n_visits):
    if v == 0:
        continue
    dates = random_dates(v, start_year, end_year, rng)
    # encounter types: office most common
    enc_types = rng.choice(["Office", "Preventive", "ED"], size=v, p=[0.72, 0.20, 0.08])
    prov_types = np.where(enc_types == "ED",
                          "Emergency",
                          rng.choice(["Primary Care", "Specialist"], size=v, p=[0.78, 0.22]))
    for d, et, pt in zip(dates, enc_types, prov_types):
        enc_rows.append(
            {
                "encounter_id": enc_id,
                "patient_id": pid,
                "encounter_date": d,
                "encounter_type": et,
                "provider_type": pt,
                "measurement_year": MEASUREMENT_YEAR,
            }
        )
        enc_id += 1

encounters = pd.DataFrame(enc_rows)

# ----------------------------
# 3) Conditions (Diabetes)
# ----------------------------
# Diabetes probability increases with age
p_diabetes = np.clip(0.04 + (ages / 90) * 0.18, 0.04, 0.22)  # 4%–22%
has_diabetes = rng.random(N_PATIENTS) < p_diabetes

cond_rows = []
cond_id = 1
for pid, diab in zip(patient_id, has_diabetes):
    if not diab:
        continue
    # diagnosis date somewhere in past 8 years
    diag_start = pd.Timestamp(f"{MEASUREMENT_YEAR-8}-01-01")
    diag_end = end_year
    diag_date = random_dates(1, diag_start, diag_end, rng).iloc[0]
    cond_rows.append(
        {
            "condition_id": cond_id,
            "patient_id": pid,
            "condition_name": "Diabetes",
            "diagnosis_date": diag_date,
            "active_flag": True,
        }
    )
    cond_id += 1

conditions = pd.DataFrame(cond_rows)

# ----------------------------
# 4) Procedures (Mammogram + HbA1c)
# ----------------------------
proc_rows = []
proc_id = 1

# Mammogram-like screening: females 50–74 more likely; still not guaranteed
# We'll allow screenings up to 2 years lookback window for measure 1.
lookback_start = pd.Timestamp(f"{MEASUREMENT_YEAR-1}-01-01")  # includes 2023-01-01
lookback_end = end_year

# HbA1c tests: mostly for diabetics, some noise
for i, row in patients.iterrows():
    pid = row["patient_id"]
    if not row["active_flag"]:
        continue

    # Mammogram: only for Female 50–74 in general population; add chance
    age = int((pd.Timestamp(f"{MEASUREMENT_YEAR}-06-30") - row["birth_date"]).days // 365.25)
    if row["sex"] == "Female" and 50 <= age <= 74:
        # chance of having at least 1 mammogram in 2-year window
        p_screen = 0.62 + (0.08 if age >= 60 else 0.0)  # modest increase
        if rng.random() < np.clip(p_screen, 0, 0.85):
            # 1–2 mammograms in window
            k = 1 + int(rng.random() < 0.12)
            dates = random_dates(k, lookback_start, lookback_end, rng)
            for d in dates:
                proc_rows.append(
                    {
                        "procedure_id": proc_id,
                        "patient_id": pid,
                        "procedure_code": "SCR-MAMMO",
                        "procedure_name": "Mammogram Screening",
                        "procedure_date": d,
                        "procedure_category": "Screening",
                    }
                )
                proc_id += 1

    # HbA1c: for diabetics, mostly within measurement year
    if pid in set(conditions["patient_id"]) if not conditions.empty else set():
        # probability of at least one HbA1c test in year
        p_hba1c = 0.70
        if rng.random() < p_hba1c:
            # 1–3 tests in measurement year
            k = rng.integers(1, 4)
            dates = random_dates(k, start_year, end_year, rng)
            for d in dates:
                proc_rows.append(
                    {
                        "procedure_id": proc_id,
                        "patient_id": pid,
                        "procedure_code": "LAB-HBA1C",
                        "procedure_name": "HbA1c Test",
                        "procedure_date": d,
                        "procedure_category": "Lab",
                    }
                )
                proc_id += 1
    else:
        # small noise rate: non-diabetics sometimes get HbA1c
        if rng.random() < 0.03:
            d = random_dates(1, start_year, end_year, rng).iloc[0]
            proc_rows.append(
                {
                    "procedure_id": proc_id,
                    "patient_id": pid,
                    "procedure_code": "LAB-HBA1C",
                    "procedure_name": "HbA1c Test",
                    "procedure_date": d,
                    "procedure_category": "Lab",
                }
            )
            proc_id += 1

procedures = pd.DataFrame(proc_rows)

# ----------------------------
# Helper: age in measurement year
# ----------------------------
def age_on_date(birth_date, asof):
    return int((asof - birth_date).days // 365.25)

patients["age_midyear"] = patients["birth_date"].apply(lambda d: age_on_date(d, pd.Timestamp(f"{MEASUREMENT_YEAR}-06-30")))

# ----------------------------
# Measures
# ----------------------------
# Measure 1: Preventive Screening Rate (Mammogram-style)
# Denominator: Female, age 50–74, active, >=1 encounter in measurement year
# Numerator: mammogram in last 2 years (2023-01-01 to 2024-12-31)
enc_in_year = encounters.groupby("patient_id")["encounter_id"].size().rename("enc_count_year")
patients_m1 = patients.merge(enc_in_year, how="left", left_on="patient_id", right_index=True)
patients_m1["enc_count_year"] = patients_m1["enc_count_year"].fillna(0).astype(int)

den_m1 = (
    (patients_m1["active_flag"])
    & (patients_m1["sex"] == "Female")
    & (patients_m1["age_midyear"].between(50, 74))
    & (patients_m1["enc_count_year"] >= 1)
)

mammos = procedures[
    (procedures["procedure_code"] == "SCR-MAMMO")
    & (procedures["procedure_date"].between(lookback_start, lookback_end))
].copy()

m1_num_patients = set(mammos["patient_id"].unique())
patients_m1["m1_den"] = den_m1
patients_m1["m1_num"] = patients_m1["patient_id"].isin(m1_num_patients) & patients_m1["m1_den"]

m1_den_count = int(patients_m1["m1_den"].sum())
m1_num_count = int(patients_m1["m1_num"].sum())
m1_rate = (m1_num_count / m1_den_count) if m1_den_count else np.nan

# Measure 2: Diabetes Management (HbA1c test in year)
# Denominator: active patients with diabetes condition active
# Numerator: HbA1c test during measurement year
diab_patients = set(conditions.loc[conditions["condition_name"] == "Diabetes", "patient_id"].unique()) if not conditions.empty else set()

hba1c = procedures[
    (procedures["procedure_code"] == "LAB-HBA1C")
    & (procedures["procedure_date"].between(start_year, end_year))
].copy()
m2_num_patients = set(hba1c["patient_id"].unique())

patients_m2 = patients.copy()
patients_m2["m2_den"] = patients_m2["active_flag"] & patients_m2["patient_id"].isin(diab_patients)
patients_m2["m2_num"] = patients_m2["m2_den"] & patients_m2["patient_id"].isin(m2_num_patients)

m2_den_count = int(patients_m2["m2_den"].sum())
m2_num_count = int(patients_m2["m2_num"].sum())
m2_rate = (m2_num_count / m2_den_count) if m2_den_count else np.nan

# ----------------------------
# Quality events table (optional)
# ----------------------------
quality_events = pd.concat(
    [
        pd.DataFrame(
            {
                "patient_id": patients_m1["patient_id"],
                "measure_name": "Preventive Screening (Mammogram-style)",
                "numerator_flag": patients_m1["m1_num"],
                "denominator_flag": patients_m1["m1_den"],
                "measurement_year": MEASUREMENT_YEAR,
            }
        ),
        pd.DataFrame(
            {
                "patient_id": patients_m2["patient_id"],
                "measure_name": "Diabetes Care (HbA1c test)",
                "numerator_flag": patients_m2["m2_num"],
                "denominator_flag": patients_m2["m2_den"],
                "measurement_year": MEASUREMENT_YEAR,
            }
        ),
    ],
    ignore_index=True,
)

# ----------------------------
# Stratification helper
# ----------------------------
def stratified_rate(df, den_col, num_col, group_cols):
    temp = df[df[den_col]].copy()
    if temp.empty:
        return pd.DataFrame()

    out = (
        temp.groupby(group_cols, dropna=False)
        .agg(
            den=("patient_id", "nunique"),
            num=(num_col, "sum")   # since num_col is boolean
        )
        .reset_index()
    )
    out["rate"] = out["num"] / out["den"]
    return out.sort_values("rate", ascending=False)

# For stratifications, attach demographic columns
m1_strat = patients_m1[["patient_id", "race", "ethnicity", "zip_code", "m1_den", "m1_num"]].copy()
m2_strat = patients_m2[["patient_id", "race", "ethnicity", "zip_code", "m2_den", "m2_num"]].copy()
# ----------------------------
# Save outputs
# ----------------------------
patients.drop(columns=["age_midyear"]).to_csv(OUT_DIR / "patients.csv", index=False)
encounters.to_csv(OUT_DIR / "encounters.csv", index=False)
conditions.to_csv(OUT_DIR / "conditions.csv", index=False)
procedures.to_csv(OUT_DIR / "procedures.csv", index=False)
quality_events.to_csv(OUT_DIR / "quality_events.csv", index=False)

# ----------------------------
# Print quick results
# ----------------------------
print("\n=== Synthetic Healthcare Quality Project ===")
print(f"Measurement year: {MEASUREMENT_YEAR}")
print(f"Patients: {len(patients):,} | Encounters: {len(encounters):,} | Conditions: {len(conditions):,} | Procedures: {len(procedures):,}")
print("\n--- Measure 1: Preventive Screening (Mammogram-style) ---")
print(f"Denominator: {m1_den_count:,} | Numerator: {m1_num_count:,} | Rate: {m1_rate:.3f}")
print("\n--- Measure 2: Diabetes Care (HbA1c test) ---")
print(f"Denominator: {m2_den_count:,} | Numerator: {m2_num_count:,} | Rate: {m2_rate:.3f}")

print("\n--- M1 Stratification by race (top 5) ---")
if m1_den_count:
    m1_by_race = stratified_rate(m1_strat, "m1_den", "m1_num", ["race"])
    print(m1_by_race.head(5).to_string(index=False))
else:
    print("No eligible denominator for M1.")

print("\n--- M2 Stratification by race (top 5) ---")
if m2_den_count:
    m2_by_race = stratified_rate(m2_strat, "m2_den", "m2_num", ["race"])
    print(m2_by_race.head(5).to_string(index=False))
else:
    print("No eligible denominator for M2.")

print(f"\nSaved CSVs to: {OUT_DIR.resolve()}\n")


=== Synthetic Healthcare Quality Project ===
Measurement year: 2024
Patients: 900 | Encounters: 2,089 | Conditions: 94 | Procedures: 258

--- Measure 1: Preventive Screening (Mammogram-style) ---
Denominator: 137 | Numerator: 83 | Rate: 0.606

--- Measure 2: Diabetes Care (HbA1c test) ---
Denominator: 88 | Numerator: 70 | Rate: 0.795

--- M1 Stratification by race (top 5) ---
    race  den  num     rate
Hispanic   26   19 0.730769
   Asian   14   10 0.714286
   Black   27   16 0.592593
   White   65   37 0.569231
   Other    5    1 0.200000

--- M2 Stratification by race (top 5) ---
    race  den  num     rate
   Black   14   12 0.857143
   White   43   36 0.837209
   Asian    9    7 0.777778
Hispanic   16   11 0.687500
   Other    6    4 0.666667

Saved CSVs to: /Users/kellykroeper/synthetic_quality_project



## Visualizations (Portfolio)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

DATA_DIR = Path("/Users/kellykroeper/synthetic_quality_project")

patients = pd.read_csv(DATA_DIR / "patients.csv")
quality = pd.read_csv(DATA_DIR / "quality_events.csv")

# Merge demographics
q_demo = quality.merge(
    patients[["patient_id", "race", "ethnicity", "sex"]],
    on="patient_id",
    how="left"
)

# --- Overall rates ---
overall = (
    q_demo[q_demo["denominator_flag"] == True]
    .groupby("measure_name", as_index=False)
    .agg(
        den=("patient_id", "nunique"),
        num=("numerator_flag", "sum")
    )
)
overall["rate"] = overall["num"] / overall["den"]
display(overall)

plt.figure()
plt.bar(overall["measure_name"], overall["rate"])
plt.xticks(rotation=20, ha="right")
plt.ylabel("Rate")
plt.title("Overall Quality Measure Rates (Synthetic Data)")
plt.tight_layout()
plt.savefig(DATA_DIR / "overall_quality_rates.png", dpi=200)
plt.show()

# --- Helper for stratified plots ---
def strat_table(df, measure_contains):
    m = df[(df["measure_name"].str.contains(measure_contains)) & (df["denominator_flag"] == True)]
    t = (
        m.groupby("race", as_index=False)
        .agg(
            den=("patient_id", "nunique"),
            num=("numerator_flag", "sum")
        )
    )
    t["rate"] = t["num"] / t["den"]
    return t.sort_values("rate", ascending=False)

# --- Preventive Screening by race ---
m1_race = strat_table(q_demo, "Preventive")
display(m1_race)

plt.figure()
plt.bar(m1_race["race"], m1_race["rate"])
plt.ylabel("Rate")
plt.title("Preventive Screening Rate by Race (Synthetic Data)")
plt.tight_layout()
plt.savefig(DATA_DIR / "preventive_screening_by_race.png", dpi=200)
plt.show()

# --- HbA1c Testing by race ---
m2_race = strat_table(q_demo, "Diabetes")
display(m2_race)

plt.figure()
plt.bar(m2_race["race"], m2_race["rate"])
plt.ylabel("Rate")
plt.title("Diabetes HbA1c Testing Rate by Race (Synthetic Data)")
plt.tight_layout()
plt.savefig(DATA_DIR / "hba1c_testing_by_race.png", dpi=200)
plt.show()

print("Saved PNG charts to:", DATA_DIR)

## Key Insights (Synthetic Example)
- Preventive Screening rate was ~61% and Diabetes HbA1c Testing rate was ~80% among eligible patients.
- Stratified views illustrate how quality reporting can surface differences across populations (note small denominators in some groups).
- This project demonstrates HEDIS-style numerator/denominator logic using a reproducible, SQL-ready analytics workflow.

## SQL Layer (SQLite)

Loaded the synthetic dataset into SQLite and queried quality measures using numerator/denominator logic. Included stratified reporting (race/ethnicity) to demonstrate equity-focused quality analytics and basic data quality checks.

In [20]:
import sqlite3
import pandas as pd
from pathlib import Path

def q(sql: str) -> pd.DataFrame:
    return pd.read_sql_query(sql, conn)

DATA_DIR = Path("/Users/kellykroeper/synthetic_quality_project")
DB_PATH = DATA_DIR / "quality_dashboard.db"

# Load CSVs
patients = pd.read_csv(DATA_DIR / "patients.csv")
encounters = pd.read_csv(DATA_DIR / "encounters.csv")
conditions = pd.read_csv(DATA_DIR / "conditions.csv")
procedures = pd.read_csv(DATA_DIR / "procedures.csv")
quality_events = pd.read_csv(DATA_DIR / "quality_events.csv")

# Create DB + write tables
conn = sqlite3.connect(DB_PATH)

patients.to_sql("patients", conn, if_exists="replace", index=False)
encounters.to_sql("encounters", conn, if_exists="replace", index=False)
conditions.to_sql("conditions", conn, if_exists="replace", index=False)
procedures.to_sql("procedures", conn, if_exists="replace", index=False)
quality_events.to_sql("quality_events", conn, if_exists="replace", index=False)

# Add a couple indexes (nice for realism)
cur = conn.cursor()
cur.execute("CREATE INDEX IF NOT EXISTS idx_qe_patient ON quality_events(patient_id);")
cur.execute("CREATE INDEX IF NOT EXISTS idx_qe_measure ON quality_events(measure_name);")
cur.execute("CREATE INDEX IF NOT EXISTS idx_patients_patient ON patients(patient_id);")
conn.commit()

print("SQLite DB created at:", DB_PATH)

SQLite DB created at: /Users/kellykroeper/synthetic_quality_project/quality_dashboard.db


In [10]:
import pandas as pd

def q(sql: str) -> pd.DataFrame:
    return pd.read_sql_query(sql, conn)

# 1) Overall measure rates (numerator/denominator)
sql_overall = """
SELECT
  measure_name,
  COUNT(CASE WHEN denominator_flag = 1 THEN 1 END) AS denominator,
  SUM(CASE WHEN denominator_flag = 1 THEN numerator_flag ELSE 0 END) AS numerator,
  ROUND(
    CAST(SUM(CASE WHEN denominator_flag = 1 THEN numerator_flag ELSE 0 END) AS REAL) /
    NULLIF(COUNT(CASE WHEN denominator_flag = 1 THEN 1 END), 0),
    3
  ) AS rate
FROM quality_events
GROUP BY measure_name
ORDER BY measure_name;
"""
overall = q(sql_overall)
overall

,measure_name,denominator,numerator,rate
0,Diabetes Care (HbA1c test),88,70,0.795
1,Preventive Screening (Mammogram-style),137,83,0.606


In [11]:
sql_m1_race = """
SELECT
  p.race,
  COUNT(*) AS denominator,
  SUM(qe.numerator_flag) AS numerator,
  ROUND(CAST(SUM(qe.numerator_flag) AS REAL) / NULLIF(COUNT(*), 0), 3) AS rate
FROM quality_events qe
JOIN patients p ON p.patient_id = qe.patient_id
WHERE qe.measure_name = 'Preventive Screening (Mammogram-style)'
  AND qe.denominator_flag = 1
GROUP BY p.race
ORDER BY rate DESC;
"""
m1_race = q(sql_m1_race)
m1_race

,race,denominator,numerator,rate
0,Hispanic,26,19,0.731
1,Asian,14,10,0.714
2,Black,27,16,0.593
3,White,65,37,0.569
4,Other,5,1,0.200


In [12]:
sql_m2_race = """
SELECT
  p.race,
  COUNT(*) AS denominator,
  SUM(qe.numerator_flag) AS numerator,
  ROUND(CAST(SUM(qe.numerator_flag) AS REAL) / NULLIF(COUNT(*), 0), 3) AS rate
FROM quality_events qe
JOIN patients p ON p.patient_id = qe.patient_id
WHERE qe.measure_name = 'Diabetes Care (HbA1c test)'
  AND qe.denominator_flag = 1
GROUP BY p.race
ORDER BY rate DESC;
"""
m2_race = q(sql_m2_race)
m2_race

,race,denominator,numerator,rate
0,Black,14,12,0.857
1,White,43,36,0.837
2,Asian,9,7,0.778
3,Hispanic,16,11,0.688
4,Other,6,4,0.667


In [13]:
sql_dq = """
SELECT
  SUM(CASE WHEN p.race IS NULL THEN 1 ELSE 0 END) AS missing_race,
  SUM(CASE WHEN p.ethnicity IS NULL THEN 1 ELSE 0 END) AS missing_ethnicity,
  SUM(CASE WHEN p.sex IS NULL THEN 1 ELSE 0 END) AS missing_sex
FROM patients p;
"""
dq = q(sql_dq)
dq

,missing_race,missing_ethnicity,missing_sex
0,0,0,0


In [14]:
sql_m1_eth = """
SELECT
  p.ethnicity,
  COUNT(*) AS denominator,
  SUM(qe.numerator_flag) AS numerator,
  ROUND(CAST(SUM(qe.numerator_flag) AS REAL) / NULLIF(COUNT(*), 0), 3) AS rate
FROM quality_events qe
JOIN patients p ON p.patient_id = qe.patient_id
WHERE qe.measure_name = 'Preventive Screening (Mammogram-style)'
  AND qe.denominator_flag = 1
GROUP BY p.ethnicity
ORDER BY rate DESC;
"""
m1_eth = q(sql_m1_eth)
m1_eth

,ethnicity,denominator,numerator,rate
0,Hispanic,36,24,0.667
1,Non-Hispanic,101,59,0.584


In [15]:
conn.close()
print("DB connection closed.")

DB connection closed.


## SQL Deep-Dive (SQLite)
- Recomputed HEDIS-style measures directly from base tables (patients, encounters, conditions, procedures)
- Produced monthly trend outputs for reporting
- Added target/threshold flags to identify underperforming strata for quality improvement

## I rebuilt numerator and denominator logic directly from raw clinical and encounter data, then validated that it matched the reporting layer using SQL.

In [21]:
MEASUREMENT_YEAR = 2024

sql_recompute = f"""
-- Recompute measures from base tables (patients/encounters/conditions/procedures)
WITH params AS (
  SELECT
    '{MEASUREMENT_YEAR}-01-01' AS year_start,
    '{MEASUREMENT_YEAR}-12-31' AS year_end,
    '{MEASUREMENT_YEAR-1}-01-01' AS lookback_start,
    '{MEASUREMENT_YEAR}-06-30' AS asof_date
),

-- M1 Denominator: active female 50–74 with >=1 encounter in measurement year
m1_den AS (
  SELECT DISTINCT p.patient_id, p.race
  FROM patients p, params par
  JOIN encounters e ON e.patient_id = p.patient_id
  WHERE p.active_flag = 1
    AND p.sex = 'Female'
    AND (CAST((julianday(par.asof_date) - julianday(p.birth_date)) / 365.25 AS INTEGER)) BETWEEN 50 AND 74
    AND date(e.encounter_date) BETWEEN par.year_start AND par.year_end
),

-- M1 Numerator: mammogram screening within 2-year window (here: 2023-01-01 to 2024-12-31)
m1_num AS (
  SELECT DISTINCT d.patient_id
  FROM m1_den d, params par
  JOIN procedures pr ON pr.patient_id = d.patient_id
  WHERE pr.procedure_code = 'SCR-MAMMO'
    AND date(pr.procedure_date) BETWEEN par.lookback_start AND par.year_end
),

-- M2 Denominator: active patients with diabetes
m2_den AS (
  SELECT DISTINCT p.patient_id, p.race
  FROM patients p
  JOIN conditions c ON c.patient_id = p.patient_id
  WHERE p.active_flag = 1
    AND c.condition_name = 'Diabetes'
    AND c.active_flag = 1
),

-- M2 Numerator: HbA1c test during measurement year
m2_num AS (
  SELECT DISTINCT d.patient_id
  FROM m2_den d, params par
  JOIN procedures pr ON pr.patient_id = d.patient_id
  WHERE pr.procedure_code = 'LAB-HBA1C'
    AND date(pr.procedure_date) BETWEEN par.year_start AND par.year_end
)

SELECT
  'Preventive Screening (Mammogram-style)' AS measure,
  (SELECT COUNT(*) FROM m1_den) AS denominator,
  (SELECT COUNT(*) FROM m1_num) AS numerator,
  ROUND(CAST((SELECT COUNT(*) FROM m1_num) AS REAL) / NULLIF((SELECT COUNT(*) FROM m1_den),0), 3) AS rate
UNION ALL
SELECT
  'Diabetes Care (HbA1c test)' AS measure,
  (SELECT COUNT(*) FROM m2_den) AS denominator,
  (SELECT COUNT(*) FROM m2_num) AS numerator,
  ROUND(CAST((SELECT COUNT(*) FROM m2_num) AS REAL) / NULLIF((SELECT COUNT(*) FROM m2_den),0), 3) AS rate;
"""
q(sql_recompute)

,measure,denominator,numerator,rate
0,Preventive Screening (Mammogram-style),137,83,0.606
1,Diabetes Care (HbA1c test),88,70,0.795


## Monthly trend reporting (time series)

In [22]:
sql_monthly = f"""
WITH params AS (
  SELECT
    '{MEASUREMENT_YEAR}-01-01' AS year_start,
    '{MEASUREMENT_YEAR}-12-31' AS year_end,
    '{MEASUREMENT_YEAR}-06-30' AS asof_date
),

months AS (
  SELECT date('{MEASUREMENT_YEAR}-01-01') AS month_start
  UNION ALL
  SELECT date(month_start, '+1 month')
  FROM months
  WHERE month_start < date('{MEASUREMENT_YEAR}-12-01')
),

m1_den AS (
  SELECT DISTINCT p.patient_id
  FROM patients p, params par
  JOIN encounters e ON e.patient_id = p.patient_id
  WHERE p.active_flag = 1
    AND p.sex = 'Female'
    AND (CAST((julianday(par.asof_date) - julianday(p.birth_date)) / 365.25 AS INTEGER)) BETWEEN 50 AND 74
    AND date(e.encounter_date) BETWEEN par.year_start AND par.year_end
),

m2_den AS (
  SELECT DISTINCT p.patient_id
  FROM patients p
  JOIN conditions c ON c.patient_id = p.patient_id
  WHERE p.active_flag = 1
    AND c.condition_name = 'Diabetes'
    AND c.active_flag = 1
)

SELECT
  'Preventive Screening (events per month)' AS series,
  strftime('%Y-%m', m.month_start) AS month,
  (SELECT COUNT(*) FROM m1_den) AS denominator,
  COUNT(DISTINCT CASE
    WHEN pr.procedure_code = 'SCR-MAMMO'
     AND date(pr.procedure_date) >= m.month_start
     AND date(pr.procedure_date) < date(m.month_start, '+1 month')
    THEN pr.patient_id END
  ) AS numerator,
  ROUND(
    CAST(COUNT(DISTINCT CASE
      WHEN pr.procedure_code = 'SCR-MAMMO'
       AND date(pr.procedure_date) >= m.month_start
       AND date(pr.procedure_date) < date(m.month_start, '+1 month')
      THEN pr.patient_id END
    ) AS REAL) / NULLIF((SELECT COUNT(*) FROM m1_den),0), 3
  ) AS rate
FROM months m
LEFT JOIN procedures pr ON pr.patient_id IN (SELECT patient_id FROM m1_den)
GROUP BY month

UNION ALL

SELECT
  'Diabetes HbA1c (events per month)' AS series,
  strftime('%Y-%m', m.month_start) AS month,
  (SELECT COUNT(*) FROM m2_den) AS denominator,
  COUNT(DISTINCT CASE
    WHEN pr.procedure_code = 'LAB-HBA1C'
     AND date(pr.procedure_date) >= m.month_start
     AND date(pr.procedure_date) < date(m.month_start, '+1 month')
    THEN pr.patient_id END
  ) AS numerator,
  ROUND(
    CAST(COUNT(DISTINCT CASE
      WHEN pr.procedure_code = 'LAB-HBA1C'
       AND date(pr.procedure_date) >= m.month_start
       AND date(pr.procedure_date) < date(m.month_start, '+1 month')
      THEN pr.patient_id END
    ) AS REAL) / NULLIF((SELECT COUNT(*) FROM m2_den),0), 3
  ) AS rate
FROM months m
LEFT JOIN procedures pr ON pr.patient_id IN (SELECT patient_id FROM m2_den)
GROUP BY month
ORDER BY series, month;
"""
monthly = q(sql_monthly)
monthly

,series,month,denominator,numerator,rate
0,Diabetes HbA1c (events per month),2024-01,88,12,0.136
1,Diabetes HbA1c (events per month),2024-02,88,9,0.102
2,Diabetes HbA1c (events per month),2024-03,88,12,0.136
3,Diabetes HbA1c (events per month),2024-04,88,14,0.159
4,Diabetes HbA1c (events per month),2024-05,88,9,0.102
5,Diabetes HbA1c (events per month),2024-06,88,13,0.148
6,Diabetes HbA1c (events per month),2024-07,88,11,0.125
7,Diabetes HbA1c (events per month),2024-08,88,9,0.102
8,Diabetes HbA1c (events per month),2024-09,88,12,0.136
9,Diabetes HbA1c (events per month),2024-10,88,11,0.125


## Target / threshold flags by race (quality improvement angle)

In [23]:
TARGET_M1 = 0.75  # screening target (example)
TARGET_M2 = 0.80  # HbA1c target (example)

sql_targets = f"""
WITH base AS (
  SELECT
    qe.measure_name,
    p.race,
    COUNT(*) AS denominator,
    SUM(qe.numerator_flag) AS numerator,
    CAST(SUM(qe.numerator_flag) AS REAL) / NULLIF(COUNT(*),0) AS rate
  FROM quality_events qe
  JOIN patients p ON p.patient_id = qe.patient_id
  WHERE qe.denominator_flag = 1
  GROUP BY qe.measure_name, p.race
)

SELECT
  measure_name,
  race,
  denominator,
  numerator,
  ROUND(rate, 3) AS rate,
  CASE
    WHEN measure_name = 'Preventive Screening (Mammogram-style)' AND rate < {TARGET_M1} THEN 'UNDER TARGET'
    WHEN measure_name = 'Diabetes Care (HbA1c test)' AND rate < {TARGET_M2} THEN 'UNDER TARGET'
    ELSE 'ON TRACK'
  END AS status
FROM base
ORDER BY measure_name, status DESC, rate ASC;
"""
q(sql_targets)

,measure_name,race,denominator,numerator,rate,status
0,Diabetes Care (HbA1c test),Other,6,4,0.667,UNDER TARGET
1,Diabetes Care (HbA1c test),Hispanic,16,11,0.688,UNDER TARGET
2,Diabetes Care (HbA1c test),Asian,9,7,0.778,UNDER TARGET
3,Diabetes Care (HbA1c test),White,43,36,0.837,ON TRACK
4,Diabetes Care (HbA1c test),Black,14,12,0.857,ON TRACK
5,Preventive Screening (Mammogram-style),Other,5,1,0.200,UNDER TARGET
6,Preventive Screening (Mammogram-style),White,65,37,0.569,UNDER TARGET
7,Preventive Screening (Mammogram-style),Black,27,16,0.593,UNDER TARGET
8,Preventive Screening (Mammogram-style),Asian,14,10,0.714,UNDER TARGET
9,Preventive Screening (Mammogram-style),Hispanic,26,19,0.731,UNDER TARGET


In [24]:
from pathlib import Path

DATA_DIR = Path("/Users/kellykroeper/synthetic_quality_project")
out = DATA_DIR / "queries.sql"

with open(out, "w") as f:
    f.write("-- Recompute measures from base tables\n")
    f.write(sql_recompute.strip() + "\n\n")
    f.write("-- Monthly trends\n")
    f.write(sql_monthly.strip() + "\n\n")
    f.write("-- Target flags by race\n")
    f.write(sql_targets.strip() + "\n")

print("Wrote:", out)

Wrote: /Users/kellykroeper/synthetic_quality_project/queries.sql
